# CityBikes

### Set-up

In [2]:
# import libraries
import requests
import pandas as pd
import time

In [3]:
# set constants
ROOT_URL = 'http://api.citybik.es/v2/networks'
CALL_DELAY = 1 # pause (in s) after each call

In [40]:
# define functions

# 'networks' functions
def getNetworks_url(filter_fields = []):
    if len(filter_fields) > 0:
        filter_fields_string = ''
        for field in filter_fields:
            filter_fields_string += '{},'.format(field)
        return ROOT_URL + '?fields={}'.format(filter_fields_string)
    return ROOT_URL

def getNetworks_df(filter_fields = []):
    response = requests.get(getNetworks_url(filter_fields)).json()
    time.sleep(CALL_DELAY)
    networks_df = pd.json_normalize(response['networks'])
    return networks_df

# 'network' functions
def getNetwork_id(city = ''):
    networks_df = getNetworks_df(['location', 'id'])
    id_df = networks_df[networks_df['location.city'] == city]
    network_id = id_df.iloc[0]['id']
    return network_id #Needs error handling for if city is a empty string

def getNetwork_df(city = ''):
    response = requests.get(ROOT_URL + '/' + getNetwork_id(city)).json()
    time.sleep(CALL_DELAY)
    network_df = pd.json_normalize(response['network']['stations'], max_level = 0)
    network_df.loc[:,'network_id'] = response['network']['id']
    network_df
    return network_df

# project specific functions
def getCityBikes_df(df = pd.DataFrame()):
    cityBikes_df = pd.DataFrame()
    cityBikes_df['network_id'] = df['network_id']
    cityBikes_df['station_id'] = df['id']
    cityBikes_df['name'] = df['name']
    cityBikes_df['bikes'] = df['empty_slots'] + df['free_bikes']
    cityBikes_df['latitude'] = df['latitude']
    cityBikes_df['longitude'] = df['longitude']
    return cityBikes_df # needs error handling for if df is empty DataFrame

### Exploring Networks

In [5]:
# parameters
filter_fields = [] # options are [name, id, href, location]

In [8]:
networks_df = getNetworks_df(filter_fields)
networks_df.head()

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,license.name,license.url,ebikes
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,Moscow,RU,55.750000,37.616667,NaN,NaN,NaN,NaN,NaN
1,[Urban Infrastructure Partner],/v2/networks/baerum-bysykkel,baerum-bysykkel,Bysykkel,Bærum,NO,59.894550,10.546343,NaN,NaN,NaN,NaN,NaN
2,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,Bicincittà,Siena,IT,43.318600,11.330600,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,NaN
3,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,Cyclopolis,Maroussi,GR,38.056872,23.808330,NaN,NaN,NaN,NaN,NaN
4,"[Groundwork, Slough Borough Council, ITS]",/v2/networks/cycle-hire-slough,cycle-hire-slough,Cycle Hire,Slough,GB,51.511350,-0.591562,NaN,NaN,NaN,NaN,NaN


In [18]:
networks_lite = networks_df.drop(columns = ['href', 'ebikes', 'source', 'gbfs_href', 'license.name', 'license.url']).rename(columns = {'id': 'network_id', 'location.latitude': 'latitude', 'location.longitude': 'longitude', 'location.city' : 'city', 'location.country' : 'country'})
networks_lite

,company,network_id,name,city,country,latitude,longitude
0,[ЗАО «СитиБайк»],velobike-moscow,Velobike,Moscow,RU,55.750000,37.616667
1,[Urban Infrastructure Partner],baerum-bysykkel,Bysykkel,Bærum,NO,59.894550,10.546343
2,[Comunicare S.r.l.],bicincitta-siena,Bicincittà,Siena,IT,43.318600,11.330600
3,[Cyclopolis Systems],cyclopolis-maroussi,Cyclopolis,Maroussi,GR,38.056872,23.808330
4,"[Groundwork, Slough Borough Council, ITS]",cycle-hire-slough,Cycle Hire,Slough,GB,51.511350,-0.591562
...,...,...,...,...,...,...,...
610,None,acces-velo-saguenay,Accès Vélo,Saguenay,CA,48.433333,-71.083333
611,"[Vilvolt CA Épinal, Communauté d'agglomération...",vilvolt,Vilvolt,Epinal,FR,48.183300,6.450800
612,"[VeloBaie, Fifteen SAS]",velobaie,VeloBaie,Saint-Brieuc,FR,48.514000,-2.765000
613,[Donkey Republic],donkey-gh,Donkey Republic - Gent,Gent,BE,51.050000,3.730300


### Exploring Network

In [36]:
# parameters
city = 'Palma'

In [37]:
getNetwork_df(city).head()

,empty_slots,extra,free_bikes,id,latitude,longitude,name,timestamp,network_id
0,10,"{'online': False, 'uid': '09'}",0,abb1d34b9bf558d40cd02e26ab1dc8b3,39.574584,2.664034,F. MANUEL HERREROS,2023-01-10T12:16:31.026000Z,bicipalma
1,18,"{'online': True, 'uid': '37'}",0,070edbcc8bdaf56dfc6821ff1c6c00fe,39.574502,2.640603,AVD ARGENTINA,2023-01-10T12:16:31.028000Z,bicipalma
2,8,"{'online': False, 'uid': '60'}",0,635e79f1cc1ab4e3efcf3aac6783e264,39.570276,2.656449,TRAVESSA BALLESTER,2023-01-10T12:16:31.029000Z,bicipalma
3,14,"{'online': False, 'uid': '61'}",0,67efc6f579c85a9609cb72429b9561fc,39.570342,2.633678,PLAÇA PONT,2023-01-10T12:16:31.030000Z,bicipalma
4,29,"{'online': False, 'uid': '63'}",0,1c2d1811ddea91526e3292521e9a6b55,39.575436,2.654550,PL. DESPANYA,2023-01-10T12:16:31.030000Z,bicipalma


#### Functionality to integrate:
- Error handling for getNetworks_id()

Explore the possibility of adding parameter fields like in Google Colab

### Acquiring project specific data

In [20]:
# parameters
city = 'Palma'

In [41]:
cityBikes_df = getCityBikes_df(getNetwork_df(city))
cityBikes_df.head()

,network_id,station_id,name,bikes,latitude,longitude
0,bicipalma,abb1d34b9bf558d40cd02e26ab1dc8b3,F. MANUEL HERREROS,10,39.574584,2.664034
1,bicipalma,070edbcc8bdaf56dfc6821ff1c6c00fe,AVD ARGENTINA,18,39.574502,2.640603
2,bicipalma,635e79f1cc1ab4e3efcf3aac6783e264,TRAVESSA BALLESTER,8,39.570276,2.656449
3,bicipalma,67efc6f579c85a9609cb72429b9561fc,PLAÇA PONT,14,39.570342,2.633678
4,bicipalma,1c2d1811ddea91526e3292521e9a6b55,PL. DESPANYA,29,39.575436,2.654550


In [22]:
cityBikes_df.to_csv('palmaBikes.csv', index = False)

#### Functionality to integrate:
- Error handling for getCityBikes()

Explore the possibility of adding parameter fields like in Google Colab